# Timing Test
My x-y image to ra-dec image program was bottlenecking at the xy-to-radec. This notebook was designed to time that method line by line to find where it was getting stuck. Turns out the icrs conversion can be done vectorwise on an np array which removes my loop timing bottleneck.

In [2]:
import math
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.time import Time
import astropy.time.core as aptime
from astropy import units as u
import numpy as np

from matplotlib.patches import Circle
from matplotlib.patches import Rectangle
import matplotlib.image as image
import matplotlib.pyplot as plot
from scipy import ndimage

import os

%load_ext line_profiler

These methods are directly lifed from the original version of coordinates.py

In [3]:
center = (256, 252)

# r - theta table.
rpoints = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 11.6]
thetapoints = [0, 3.58, 7.17, 10.76, 14.36, 17.98, 21.62, 25.27, 28.95, 32.66, 36.40, 40.17, 43.98, 47.83, 51.73, 55.67, 59.67, 63.72, 67.84, 72.03, 76.31, 80.69, 85.21, 89.97,90]

# Returns a tuple of form (alt, az)
def xy_to_altaz(x,y):

    # Point adjusted based on the center being at... well... the center. And not the top left. In case you were confused.
    # Y is measured from the top stop messing it up.
    pointadjust = (x - center[0], center[1] - y)

    # We use -x here because the E and W portions of the image are flipped
    az = math.atan2(-(pointadjust[0]),(pointadjust[1]))

    # For same reason as -x, we use > 0 here
    # atan2 ranges from -pi to pi so we have to shift the -pi to 0 parts up 2 pi.
    if pointadjust[0] > 0:
        az += 2 * math.pi

    az = math.degrees(az)

    # Pythagorean thereom boys.
    r = math.sqrt(pointadjust[0]**2 + pointadjust[1]**2)

    # For now if r is on the edge of the circle or beyond we'll have it just be 0 degrees. (Up from horizontal)
    if r > 240:
        alt = 0
    else:
        #r = 238 # Debug line
        r = r * 11.6 / 240 # Magic pixel to mm conversion rate

        # 90- turns the angle from measured from the vertical to measured from the horizontal.
        # This interpolates the value from the two on either side of it.
        alt = 90 - np.interp(r, xp = rpoints, fp = thetapoints)

    # Az correction
    az = az + .94444
    
    return (alt,az)

# Returns a tuple of form (ra, dec)
def altaz_to_radec(alt, az, time):
    assert type(time) is aptime.Time, "Time should be an astropy Time Object."

    # This is the latitutde/longitude of the camera
    cameraloc = (31.959417, -111.598583)

    cameraearth = EarthLocation(lat = cameraloc[0] * u.deg, lon = cameraloc[1] * u.deg, height = 2120 * u.m)

    altazcoord = SkyCoord(alt = alt * u.deg, az = az * u.deg, frame = 'altaz', obstime = time, location = cameraearth)
    radeccoord = altazcoord.icrs

    return (radeccoord.ra.degree, radeccoord.dec.degree)

# Returns a tuple of form (alt, az)
def radec_to_altaz(ra, dec, time):
    assert type(time) is aptime.Time, "Time should be an astropy Time Object."

    # This is the latitutde/longitude of the camera
    cameraloc = (31.959417, -111.598583)

    cameraearth = EarthLocation(lat = cameraloc[0] * u.deg, lon = cameraloc[1] * u.deg, height = 2120 * u.meter)


    # Creates the SkyCoord object
    radeccoord = SkyCoord(ra = ra, dec = dec, unit = 'deg', obstime = time, location = cameraearth, frame = 'icrs', temperature = 5 * u.deg_C, pressure = 78318 * u.Pa)

    # Transforms
    altazcoord = radeccoord.transform_to('altaz')

    return (altazcoord.alt.degree, altazcoord.az.degree)

# Returns a tuple of form (x,y)
def altaz_to_xy(alt, az):
    # Approximate correction (due to distortion of lens?)
    az = az - .94444

    #print(az)

    # Reverse of r interpolation
    r = np.interp(90 - alt, xp = thetapoints, fp = rpoints)

    r = r * 240 / 11.6 # mm to pixel rate

    # Remember angle measured from vertical so sin and cos are swapped from usual polar.
    # These are x,ys with respect to a zero.
    x = -1 * r * math.sin(math.radians(az))
    y = r * math.cos(math.radians(az))

    # y is measured from the top!
    pointadjust = (x + center[0], center[1] - y)

    return pointadjust

# Returns a tuple of form (x,y)
# Time must be an astropy Time object.
# Please.
def radec_to_xy(ra, dec, time):
    altaz = radec_to_altaz(ra, dec, time)
    x,y = altaz_to_xy(altaz[0], altaz[1])
    return galactic_conv(x,y,altaz[1])

# Returns a tuple of form (ra,dec)
# Time must be an astropy Time object.
# Please.
def xy_to_radec(x,y,time):
    altaz = xy_to_altaz(x,y)
    x,y = camera_conv(x,y,altaz[1])
    
    altaz = xy_to_altaz(x,y)
    return altaz_to_radec(altaz[0], altaz[1], time)

def timestring_to_obj(date, filename):
    # Add the dashes
    formatted = date[:4] + '-' + date[4:6] + '-' + date[6:]

    # Extracts the time from the file name. File names seem to be machine generated so this should not break.
    # Hopefully.
    time = filename[4:6] + ':' + filename[6:8] + ':' + filename[8:10]

    formatted = formatted + ' ' + time

    return Time(formatted)

# Converts from galactic x,y, expected az to camera x,y, actual az
def galactic_conv(x,y,az):

    r = math.sqrt(x**2 + y**2)
    az = az - .94444
    #print(r)
    # These magic numbers found using the genius method of "lots of trial and error."
    # Just kidding, I did a chi-squared analysis with a bunch of different models and this was the best I came up with.
    r = r + 2.369 * math.cos(math.radians(0.997 * (az - 42.088))) + 0.699
    az = az + 0.716 * math.cos(math.radians(1.015 * (az + 31.358))) - 0.181
    
    x = -1 * r * math.sin(math.radians(az))
    y = r * math.cos(math.radians(az))
    
    return (x,y)

# Converts from camera r,az to galactic r,az
def camera_conv(x,y,az):
    r = math.sqrt(x**2 + y**2)
    
    # You might think that this should be + but actually no. See next comment.
    az = az - .94444 #- 0.286375
    
    az = az - 0.731 * math.cos(math.radians(0.993 * (az + 34.5))) + 0.181
    r = r - 2.358 * math.cos(math.radians(0.99 * (az - 40.8))) - 0.729
    
    x = -1 * r * math.sin(math.radians(az))
    y = r * math.cos(math.radians(az))
    
    return (x,y)

Line by line profiling of the methods. Eventually I realized that xy to altaz can be performed vectorwise instead of in a loop.

In [28]:
#%%timeit 

time = timestring_to_obj('20170822','r_ut044514s10080.png')

x = 200
y = 200

alt = np.zeros(50)
az = np.zeros(50)

print(altaz_to_radec(alt,az,time))
#for x in range(100,151):
    #for y in range(100,151):
    
%lprun -f altaz_to_radec altaz_to_radec(alt,az,time)

(array([ 110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143,  110.03723143,  110.03723143,
        110.03723143,  110.03723143]), array([ 58.07766402,  58.07766402,  58.07766402,  58.07766402,
        58.07766402,  58.07766402,  58.07766402,  58.07766402,
        58.07766402,  58.07766